<a href="https://colab.research.google.com/github/uasap/Analit/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Untitled11_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

будем работать только с частью нашего набора данных, разбитой на 4 категории из 20 возможных:

список файлов, совпадающих с нужными категориями


In [ ]:
categories = ['alt.atheism', 'soc.religion.christian',  'comp.graphics', 'sci.med']

from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

twenty_train.target_names
['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

Возвращаемый набор данных — это scikit-learn совокупность: одномерный контейнер с полями, которые могут интерпретироваться как ключи в словаре python (dict keys), проще говоря — как признаки объекта (object attributes). Например, target_names содержит список названий запрошенных категорий:

выведем на печать первые строки из первого загруженного файла:

In [ ]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))
print(twenty_train.target_names[twenty_train.target[0]])


From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
comp.graphics


Алгоритмы для обучения с учителем (контролируемого обучения) требуют, чтобы у каждого документа в обучающей выборке была помета определенной категории. В нашем случае, категория — это название новостной выборки

 название категории:

In [ ]:
twenty_train.target[:10]
for t in twenty_train.target[:10]:
     print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


Извлечение характерных признаков из текстовых файлов

Чтобы использовать машинное обучение на текстовых документах, первым делом, нужно перевести текстовое содержимое в числовой вектор признаков.
«Мешок слов» (набор слов)

Наиболее интуитивно понятный способ сделать описанное выше преобразование — это представить текст в виде набора слов:
приписать уникальный целочисленный индекс каждому слову, появляющемуся в документах в обучающей выборке (например, построив словарь из слов с целочисленными индексами).
для каждого документа #i посчитать количество употреблений каждого слова w и сохранить его (количество) в X[i, j]. Это будет значение признака #j, где j — это индекс слова w в словаре.

Предобработка текста, токенизация и отфильтровывание стоп-слов включены в состав высоко уровневого компонента, который позволяет создать словарь характерных признаков и перевести документы в векторы признаков:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

CountVectorizer поддерживает подсчет N-грам слов или последовательностей символов. Векторизатор строит словарь индексов признаков:

In [ ]:
count_vect.vocabulary_.get(u'algorithm')

4690

в длинных документах среднее количество словоупотреблений будет выше, чем в коротких, даже если они посвящены одной теме.
Чтобы избежать этих потенциальных несоответствий, достаточно разделить количество употреблений каждого слова в документе на общее количество слов в документе. Этот новый признак называется tf — Частота термина.
Следующее уточнение меры tf — это снижение веса слова, которое появляется во многих документах в корпусе, и отсюда является менее информативным, чем те, которые используются только в небольшой части корпуса. Примером низко ифнормативных слов могут служить служебные слова, артикли, предлоги, союзы и т.п.
Это снижение называется tf–idf, что значит “Term Frequency times Inverse Document Frequency” (обратная частота термина).

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

обучим классификатор предсказывать категорию текста.  начнем с Наивного Байесовского классификатора. scikit-learn включает в себя несколько вариантов этого классификатора. Самый подходящий для подсчета слов — это его поли номинальный вариант:

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

нужно извлечь фичи (характерные признаки), используя почти такую же последовательность как и ранее. Разница состоит в том, используется метод transform вместо fit_transform из transformers, так как они уже были применены к нашей обучающей выборке:

In [ ]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


Чтобы с цепочкой vectorizer => transformer => classifier было проще работать, в scikit-learn есть класс Pipeline, который функционирует как составной (конвейерный) классификатор:


In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

In [ ]:
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

Оценка точности прогноза модели

In [ ]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
     categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

с помощью линейного метода опорных векторов (support vector machine (SVM)).

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3,  random_state=42)),
 ])
_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9114513981358189

In [ ]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
     target_names=twenty_test.target_names))



                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.88       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.96      0.88      0.92       396
soc.religion.christian       0.89      0.96      0.92       398

              accuracy                           0.91      1502
             macro avg       0.92      0.91      0.91      1502
          weighted avg       0.92      0.91      0.91      1502



In [ ]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[259,  10,  12,  38],
       [  3, 380,   2,   4],
       [  6,  36, 349,   5],
       [  5,  10,   2, 381]])

Некоторые параметры мы уже посчитали, такие как use_idf в функции TfidfTransformer.
Вместо поиска параметров различных компонентов в цепи, можно запустить поиск (методом полного перебора ) лучших параметров в сетке возможных значений. Мы опробовали все классификаторы на словах или биграммах, с или без idf, с штрафными параметрами 0,01 и 0,001 для SVM (метода опорных векторов):



In [ ]:
import sklearn
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),
 }



можем запустить grid search, чтобы попробовать все комбинации параметров параллельно с параметром n_jobs. Если мы зададим этому параметру значение -1, grid search определит, как много ядер установлено, и задействует их все:

In [ ]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])


In [ ]:
idx_pr = gs_clf.predict(['God is love'])
twenty_train.target_names[idx_pr[0]]

'soc.religion.christian'